<a href="https://colab.research.google.com/github/DhruvSaraogi/OOP-Project/blob/main/notebooks/Chest_xray_localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Save a copy of this Notebook before execution.

Check GPU configuration.
In this project Nvidia tesla P100 GPU was used.

In [1]:
!nvidia-smi

Tue Jan  7 16:26:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Clone the Repository

In [2]:
!git clone https://github.com/gavin-d26/Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention.git

Cloning into 'Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 53 (delta 23), reused 36 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 803.62 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (23/23), done.


Create a kaggle API token and upload the downloaded kaggle.json to the content directory.

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dhruvsaraogi","key":"7790112e0390feac8279a90bbd9ceef9"}'}

Create a directory named kaggle that will store the kaggle.json file,
and display list of kaggle datasets.


In [4]:
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
stealthtechnologies/predict-student-performance-dataset          Predict Student Performance                          12KB  2024-12-26 12:57:04           1972         51  1.0              
anandshaw2001/mobile-apps-screentime-analysis                    Mobile Apps ScreenTime Analysis                       2KB  2024-12-31 18:20:51            844         24  1.0              
oktayrdeki/heart-disease                                         Heart Disease                                       568KB  2024-12-29 13:26:49           1509         37  1.0              
ankushpanday1/heart-attack-in-youth-of-india           

Create a directory to save models

In [5]:
!mkdir saved_models

1.   Change current directory to project directory.
2.   Download nih-chest-xrays dataset as a data.zip file into project directory.
1.   Unzip the data.zip file.










In [6]:
%cd Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention
!mkdir data
%cd data
!kaggle datasets download -d nih-chest-xrays/data
!unzip \*.zip  && rm *.zip
%cd ..

Streaming output truncated to the last 5000 lines.
  inflating: images_004/images/00009059_000.png  
  inflating: images_004/images/00009060_000.png  
  inflating: images_004/images/00009061_000.png  
  inflating: images_004/images/00009061_001.png  
  inflating: images_004/images/00009061_002.png  
  inflating: images_004/images/00009061_003.png  
  inflating: images_004/images/00009061_004.png  
  inflating: images_004/images/00009061_005.png  
  inflating: images_004/images/00009061_006.png  
  inflating: images_004/images/00009062_000.png  
  inflating: images_004/images/00009063_000.png  
  inflating: images_004/images/00009064_000.png  
  inflating: images_004/images/00009065_000.png  
  inflating: images_004/images/00009066_000.png  
  inflating: images_004/images/00009067_000.png  
  inflating: images_004/images/00009067_001.png  
  inflating: images_004/images/00009067_002.png  
  inflating: images_004/images/00009067_003.png  
  inflating: images_004/images/00009067_004.png  

Install necessary packages

In [12]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 MB 8.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


Login to Weights and Biases (wandb) for experiment tracking.

In [10]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html


Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 618.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires torch>=1.13.0, but you have torch 1.11.0+cu113 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.11.0+cu113 which is incompatible.


In [11]:
!pip install opencv


ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


In [8]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Next, in weights and biases, create a new project titled "chest-xray-localization".

Run the trainpcm.py script with the following options.

1.   --savedirectory : Path to save directory
2.   --model : Type of model (resnet50, resnet50PCM, efficientnetb4, efficientnetb4PCM)
3.   --name : Name of current run
4.   --project : Name of wandb project
5.   --entity : Name of wandb entity (username) (for more info [link](https://docs.wandb.ai/ref/python/init))



In [9]:
%cd /content/Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention
!python scripts/trainpcm.py --savedirectory /content/saved_models --model resnet50PCM --name resnet50PCM_v1 --project chest-xray-localization --entity YOUR_WANDB_USERNAME

/content/Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention
Traceback (most recent call last):
  File "/content/Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention/scripts/trainpcm.py", line 6, in <module>
    import torchmetrics as tm
ModuleNotFoundError: No module named 'torchmetrics'


During Training, two versions of the model are saved. Final (_FW_) and Checkpoint (_CW_). Checkpoint refers to the model with highest Validation AUROC Score.
Go to wandb to view results.


1.   For classification, see AUROC_MACRO_TEST_Checkpoint in summary section.
2.   For localization, see table titled RUNNAME-CW-default/pcm/pcm-rv_Checkpoint

